In [11]:
import sys
import time
import cv2
import mediapipe as mp

# ——— PARAMETERS ———
SCALE = 0.5         # downscale factor for inference
SKIP_FRAMES = 1     # skip this many frames between each processed frame
LINE_THICKNESS = 2
CIRCLE_RADIUS = 5
FONT_SCALE = 0.8
FONT_THICKNESS = 1

def draw_custom_landmarks(image, landmarks, connections):
    """
    Draw thicker white lines and larger indexed circles/text.
    Coordinates are mapped to the full-resolution frame.
    """
    h, w, _ = image.shape

    # thicker lines
    for start_idx, end_idx in connections:
        lm0 = landmarks[start_idx]
        lm1 = landmarks[end_idx]
        if lm0.visibility > 0.5 and lm1.visibility > 0.5:
            x1, y1 = int(lm0.x * w), int(lm0.y * h)
            x2, y2 = int(lm1.x * w), int(lm1.y * h)
            cv2.line(image, (x1, y1), (x2, y2),
                     (255, 255, 255), thickness=LINE_THICKNESS)

    # larger circles + numbers
    for idx, lm in enumerate(landmarks):
        if lm.visibility > 0.5:
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(image, (cx, cy),
                       CIRCLE_RADIUS, (255, 255, 255), -1)
            cv2.putText(image, str(idx),
                        (cx + CIRCLE_RADIUS + 2, cy - CIRCLE_RADIUS - 2),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        FONT_SCALE, (255, 255, 255), FONT_THICKNESS)

def main():
    video_path = r"C:\Users\VA\run.mov"  # adjust if needed

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Failed to open video: {video_path}")
        sys.exit(1)
    print(f"✅ Opened video: {video_path}")

    # full-resolution size
    full_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    full_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # setup Mediapipe
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    # prepare window at exact native size
    window = "Pose Detection"
    cv2.namedWindow(window, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window, full_w, full_h)
    try:
        cv2.setWindowProperty(window,
                              cv2.WND_PROP_ASPECT_RATIO,
                              cv2.WINDOW_KEEPRATIO)
    except:
        pass

    frame_idx = 0
    t_last = time.time()

    while True:
        ret, frame = cap.read()

        # if we hit the end, loop back to start
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frame_idx = 0
            continue

        # skip frames for speed
        if frame_idx % (SKIP_FRAMES + 1) != 0:
            frame_idx += 1
            continue

        # downscale for inference
        small = cv2.resize(
            frame,
            (int(full_w * SCALE), int(full_h * SCALE))
        )

        # pose detection
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)

        # draw on full-resolution frame
        if results.pose_landmarks:
            draw_custom_landmarks(
                frame,
                results.pose_landmarks.landmark,
                mp_pose.POSE_CONNECTIONS
            )

        # show and check for quit
        cv2.imshow(window, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🛑 Exit requested by user.")
            break

        # optional: print FPS every 60 processed frames
        frame_idx += 1
        if frame_idx % 60 == 0:
            fps = 60 / (time.time() - t_last)
            print(f"≈ {fps:.1f} FPS")
            t_last = time.time()

    cap.release()
    cv2.destroyAllWindows()
    pose.close()

if __name__ == "__main__":
    main()

✅ Opened video: C:\Users\VA\run.mov
🛑 Exit requested by user.


### Include CSV with XYZ Coordinates

In [ ]:
# save_pose_data_filtered_10frames.py

import os
import sys
import cv2
import csv
import numpy as np
import mediapipe as mp

# ——— PARAMETERS ———
DESIRED_LANDMARKS = {
    "LEFT_HIP", "RIGHT_HIP",
    "LEFT_KNEE", "RIGHT_KNEE",
    "LEFT_ANKLE", "RIGHT_ANKLE",
    "LEFT_HEEL", "RIGHT_HEEL",
    "LEFT_FOOT_INDEX", "RIGHT_FOOT_INDEX"
}
VISIBILITY_THRESH = 0.5
TARGET_FRAMES = 10

STATIONARY_THRESH = 0.005
WALKING_THRESH    = 0.05

def classify_movement(speed):
    if speed < STATIONARY_THRESH:
        return "stationary"
    elif speed < WALKING_THRESH:
        return "walking"
    else:
        return "running"

def save_pose_data(video_path, output_csv="pose_walk.csv"):
    # Backup old CSV if it exists
    if os.path.exists(output_csv):
        os.rename(output_csv, output_csv + ".bak")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Failed to open video: {video_path}")
        sys.exit(1)

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Frame", "Name",
            "X", "Y", "Z", "Score",
            "X_diff", "Y_diff", "Z_diff",
            "Speed", "Acceleration", "Movement"
        ])

        prev_data = {}               # store previous valid-frame coords+speed per landmark
        valid_frames = 0
        frame_idx = 0

        while valid_frames < TARGET_FRAMES:
            ret, frame = cap.read()
            if not ret:
                print(f"⚠️ Reached end of video after {frame_idx} frames, only {valid_frames} valid frames found.")
                break

            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            # Check if all desired landmarks are visible this frame
            if results.pose_landmarks:
                lm_list = results.pose_landmarks.landmark
                present = {
                    mp_pose.PoseLandmark(i).name
                    for i, lm in enumerate(lm_list)
                    if lm.visibility >= VISIBILITY_THRESH
                }
                if DESIRED_LANDMARKS.issubset(present):
                    # This is a valid frame: write 10 rows
                    for i, lm in enumerate(lm_list):
                        name = mp_pose.PoseLandmark(i).name
                        if name not in DESIRED_LANDMARKS:
                            continue

                        x, y, z = lm.x, lm.y, lm.z
                        score = lm.visibility

                        # Compute diffs relative to last valid frame
                        if name in prev_data:
                            x_diff = x - prev_data[name]["x"]
                            y_diff = y - prev_data[name]["y"]
                            z_diff = z - prev_data[name]["z"]
                            speed  = float(np.linalg.norm([x_diff, y_diff, z_diff]))
                            acceleration = speed - prev_data[name]["speed"]
                        else:
                            x_diff = y_diff = z_diff = speed = acceleration = 0.0

                        movement = classify_movement(speed)

                        writer.writerow([
                            frame_idx, name,
                            f"{x:.6f}", f"{y:.6f}", f"{z:.6f}", f"{score:.6f}",
                            f"{x_diff:.6f}", f"{y_diff:.6f}", f"{z_diff:.6f}",
                            f"{speed:.6f}", f"{acceleration:.6f}", movement
                        ])

                        # Update for next valid frame
                        prev_data[name] = {"x": x, "y": y, "z": z, "speed": speed}

                    valid_frames += 1

            frame_idx += 1

    cap.release()
    pose.close()
    print(f"✅ Saved {valid_frames} valid frames (×10 landmarks) to {output_csv}")

if __name__ == "__main__":
    video_path = r"C:\Users\VA\walk.mov"  # adjust if needed
    save_pose_data(video_path)

### Segregated Task

In [6]:
import sys
import time
import cv2
import mediapipe as mp
import numpy as np

# ——— PARAMETERS ———
SCALE = 0.5            # downscale factor for inference
SKIP_FRAMES = 1        # skip this many frames between each processed frame
LINE_THICKNESS = 2
CIRCLE_RADIUS = 5
FONT_SCALE = 0.8
FONT_THICKNESS = 1

def draw_custom_landmarks(image, landmarks, connections):
    """
    Draw thicker white skeleton lines and indexed circles/text
    onto a blank image.
    """
    h, w, _ = image.shape

    # Draw skeleton lines
    for start_idx, end_idx in connections:
        lm0 = landmarks[start_idx]
        lm1 = landmarks[end_idx]
        if lm0.visibility > 0.5 and lm1.visibility > 0.5:
            x1, y1 = int(lm0.x * w), int(lm0.y * h)
            x2, y2 = int(lm1.x * w), int(lm1.y * h)
            cv2.line(image, (x1, y1), (x2, y2),
                     (255, 255, 255), thickness=LINE_THICKNESS)

    # Draw joints and indices
    for idx, lm in enumerate(landmarks):
        if lm.visibility > 0.5:
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(image, (cx, cy), CIRCLE_RADIUS, (255, 255, 255), -1)
            cv2.putText(image, str(idx),
                        (cx + CIRCLE_RADIUS + 2, cy - CIRCLE_RADIUS - 2),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        FONT_SCALE, (255, 255, 255), FONT_THICKNESS)

def main():
    video_path = r"C:\Users\VA\walk.mov"  # adjust if needed
    output_path = "skeletonsonly_walk.mp4"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Failed to open video: {video_path}")
        sys.exit(1)

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    full_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    full_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Setup video writer (MP4)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (full_w, full_h))

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    window = "Skeleton Only"
    cv2.namedWindow(window, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window, full_w, full_h)
    try:
        cv2.setWindowProperty(window,
                              cv2.WND_PROP_ASPECT_RATIO,
                              cv2.WINDOW_KEEPRATIO)
    except:
        pass

    frame_idx = 0
    t_last = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            # loop video
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frame_idx = 0
            continue

        # skip frames for speed
        if frame_idx % (SKIP_FRAMES + 1) != 0:
            frame_idx += 1
            continue

        # downscale for inference
        small = cv2.resize(frame, (int(full_w * SCALE), int(full_h * SCALE)))

        # pose detection
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)

        # prepare black canvas
        skeleton_frame = np.zeros((full_h, full_w, 3), dtype=np.uint8)

        # draw skeleton if found
        if results.pose_landmarks:
            draw_custom_landmarks(
                skeleton_frame,
                results.pose_landmarks.landmark,
                mp_pose.POSE_CONNECTIONS
            )

        # show and write
        cv2.imshow(window, skeleton_frame)
        writer.write(skeleton_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🛑 Exit requested by user.")
            break

        frame_idx += 1
        if frame_idx % 60 == 0:
            fps_est = 60 / (time.time() - t_last)
            print(f"≈ {fps_est:.1f} FPS")
            t_last = time.time()

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    pose.close()
    print(f"✅ Saved skeleton-only video to {output_path}")

if __name__ == "__main__":
    main()

🛑 Exit requested by user.
✅ Saved skeleton-only video to skeletonsonly_walk.mp4
